In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [1]:
import sys
import os

In [2]:
current_dir = os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_dir, os.pardir))
other_folder = os.path.join(parent_dir, 'app')
sys.path.append(other_folder)

In [3]:
from lib import *

In [4]:
shop_sales = pd.read_csv('../app/test_data/shop_sales.csv')
shop_sales_dates = pd.read_csv('../app/test_data/shop_sales_dates.csv')
shop_sales_prices = pd.read_csv('../app/test_data/shop_sales_prices.csv')

In [5]:
df = merge_files_to_dataset(shop_sales, shop_sales_prices, shop_sales_dates)

In [7]:
df = summ_sales_data(df, 'date', granularity = 'Day')

In [8]:
df = df_encoding(df)

In [9]:
data_prediction = get_preds(df, ['STORE_1_714'], 1)

Could not find the number of physical cores for the following reason:
found 0 physical cores < 1
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  File "c:\Users\vitya\OneDrive\Рабочий стол\venvs\hardml\lib\site-packages\joblib\externals\loky\backend\context.py", line 245, in _count_physical_cores
    raise ValueError(


запуск первого шага
new var_for_add ! month_lag_1
едем дальше
в итоге получили список ['month_lag_1']
запуск первого шага
new var_for_add ! cnt_EMA_3_lag_1
new var_for_add ! cnt_WMA_3_lag_1
едем дальше
в итоге получили список ['month_lag_1', 'cnt_WMA_3_lag_1']
запуск первого шага
new var_for_add ! weekday_Friday_lag_1
едем дальше
в итоге получили список ['month_lag_1', 'cnt_WMA_3_lag_1', 'weekday_Friday_lag_1']
запуск первого шага
new var_for_add ! year_lag_1
едем дальше
в итоге получили список ['month_lag_1', 'cnt_WMA_3_lag_1', 'weekday_Friday_lag_1', 'year_lag_1']
запуск первого шага
new var_for_add ! weekday_Sunday_lag_1
едем дальше
в итоге получили список ['month_lag_1', 'cnt_WMA_3_lag_1', 'weekday_Friday_lag_1', 'year_lag_1', 'weekday_Sunday_lag_1']
запуск первого шага
мы сошлись
['month_lag_1', 'cnt_WMA_3_lag_1', 'weekday_Friday_lag_1', 'year_lag_1', 'weekday_Sunday_lag_1']
0.424



A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [10]:
data_prediction

,month_lag_1,cnt_WMA_3_lag_1,weekday_Friday_lag_1,year_lag_1,weekday_Sunday_lag_1,mean,cnt_SMA_3_lag_1,item_id,date,cnt,model_prediction
80000,12.0,12.000000,0.0,2015.0,0.0,16.5,13.333333,STORE_1_714,2015-12-17,20.0,16.257999
80001,12.0,15.333333,0.0,2015.0,0.0,16.5,13.666667,STORE_1_714,2015-12-18,20.0,19.190921
80002,12.0,18.500000,1.0,2015.0,0.0,16.5,17.000000,STORE_1_714,2015-12-19,34.0,22.528674
80003,12.0,27.000000,0.0,2015.0,0.0,16.5,24.666667,STORE_1_714,2015-12-20,12.0,23.870809
80004,12.0,20.666667,0.0,2015.0,1.0,16.5,22.000000,STORE_1_714,2015-12-21,10.0,18.217096
80005,12.0,14.666667,0.0,2015.0,0.0,16.5,18.666667,STORE_1_714,2015-12-22,26.0,19.190921
80006,12.0,18.333333,0.0,2015.0,0.0,16.5,16.000000,STORE_1_714,2015-12-23,25.0,19.138627
80007,12.0,22.833333,0.0,2015.0,0.0,16.5,20.333333,STORE_1_714,2015-12-24,15.0,20.393423
80008,12.0,20.166667,0.0,2015.0,0.0,16.5,22.000000,STORE_1_714,2015-12-25,1.0,19.278077
80009,12.0,9.666667,1.0,2015.0,0.0,16.5,13.666667,STORE_1_714,2015-12-26,14.0,18.929496


In [38]:
result = evaluate_predictions(data_prediction)

NameError: name 'evaluate_predictions' is not defined

In [ ]:
result